In [1]:
import sys
sys.path.append('../scripts')

In [2]:
import pandas as pd
import spacy
import os
from pathlib import Path
from xml.dom import minidom
from charsplit import Splitter
from rules.ellipticCompounds import exchangeLemmas
from evaluation import error_analysis, get_scores

nlp = spacy.load("de_core_news_lg")

In [3]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path=Path('..'), job_name='foo', version_base='1.1')
config = compose(config_name='experiment.yaml')

In [4]:
tmp_path = Path('../tmp/baseline_files')
tmp_path.mkdir(exist_ok=True, parents=True)

In [5]:
path_xml_file = tmp_path / 'Original.xml'
path_output_file = tmp_path / 'Output.xml'
path_gertwol_file = tmp_path / 'gertwol.txt'
path_word_freqs = tmp_path / 'wordFreqs.txt'
tagged_path = '../compoundListTagged.txt'

path_tokens = Path('..') / config.data.ggponc_plain_text

# Load the Data

In [9]:
ellipses = pd.read_csv(Path('..') / config.data.cnf_tsv_path, sep='\t')
not_ellipses = pd.read_csv(Path('..') / 'data/ellipses/ggponc_no_ellipses_small.tsv', sep='\t')

not_ellipses['full_resolution'] = not_ellipses.raw_sentence
#ellipses = ellipses[ellipses.split == 'test']
#not_ellipses = not_ellipses[not_ellipses.split == 'test']

# Train and Test split?

data = pd.concat([ellipses[['file', 'sentence_id', 'raw_sentence', 'full_resolution']], not_ellipses[['file', 'sentence_id', 'raw_sentence', 'full_resolution']]]).sort_values(by='file').reset_index(drop=True)

In [10]:
data.raw_sentence.str.split(' ').apply(len).sum()

166870

In [11]:
len(data[['file', 'sentence_id']].drop_duplicates())

7696

In [12]:
data.shape

(7696, 4)

In [13]:
data.head()

,file,sentence_id,raw_sentence,full_resolution
0,00_mundhoehlenkarzinom_0002.tsv,1,Hauptrisikofaktoren für das Auftreten eines Mu...,Hauptrisikofaktoren für das Auftreten eines Mu...
1,00_mundhoehlenkarzinom_0002.tsv,2,Bei chronischem Tabak- oder Alkoholabusus ist ...,Bei chronischem Tabakabusus oder Alkoholabusus...
2,00_mundhoehlenkarzinom_0005.tsv,12,Die hohe Diskrepanz zwischen p16 Immunhistoche...,Die hohe Diskrepanz zwischen p16 Immunhistoche...
3,00_mundhoehlenkarzinom_0005.tsv,18,Die Prävalenz von HPV16 E6 oder E7 Antikörpern...,Die Prävalenz von HPV16 E6 Antikörpern oder HP...
4,00_mundhoehlenkarzinom_0005.tsv,10,Der Nachweis von Hochrisiko HPV-DNA in der Mun...,Der Nachweis von Hochrisiko HPV-DNA in der Mun...


# Create XML Document

In [14]:
root = minidom.Document()

xml = root.createElement('corpus')
xml.setAttribute('id', 'corpus')
root.appendChild(xml)

for i, f in enumerate(data[['file', 'raw_sentence']].groupby('file').first().index.to_list()):
    file = root.createElement('file')
    file.setAttribute('n', str(i))
    xml.appendChild(file)

    div = root.createElement('div')
    file.appendChild(div)

    for j, s in enumerate(data[data['file']==f]['raw_sentence']):
        sentence = root.createElement('s')
        sentence.setAttribute('lang', 'de')
        sentence.setAttribute('n', f'{i}-{j}')
        div.appendChild(sentence)

        doc = nlp(s)
        for k, token in enumerate(doc):
            word = root.createElement('w')
            word.setAttribute('lemma', token.lemma_)
            word.setAttribute('n', f'{i}-{j}-{k}')
            word.setAttribute('pos', token.tag_)
            word.setAttribute('whitespace', token.whitespace_)
            sentence.appendChild(word)
            word.appendChild(root.createTextNode(token.text))

with open(path_xml_file, "wb") as f:
    f.write(root.toprettyxml(indent ="\t", encoding="utf-8"))

# Produce Gertwol-like List and Gather Word Frequencies

In [15]:
freq = {}
dict = {}
splitter = Splitter()
special_characters = "!@#$%^&*()-+?_=,<>/.[] "

for filename in os.listdir(path_tokens):
    with open(path_tokens / filename , encoding='utf8') as f:
        for word in f.readlines():
            word = word[:-1]

            # Gertwol Split
            split = splitter.split_compound(word)[0]
            if split[0] != 0:
                dict[word] = '#'.join([split[1]] + [subword.lower() for subword in split[2:]])

            # Word Freq
            if word in special_characters:
                continue
            if word in freq:
                freq[word] += 1
            else:
                freq[word] = 1

with open(path_gertwol_file, 'w', encoding='utf-8') as f:
    for key, value in dict.items():
        f.write(f'{key}\t{value}\n')

with open(path_word_freqs, 'w', encoding='utf-8') as f:
    for key, value in freq.items():
        f.write(f'{key}\t{value}\n')

# Run Aepli Baseline

In [16]:
exchangeLemmas(path_xml_file, path_output_file, path_gertwol_file, path_word_freqs, tagged_path)

Alkohol#abusus
Alkohol#abusus
Hals#tumoren
Farb#markierung
Funktions#erhalt
Schluck#funktion
Gesichts#chirurgen
Gesichts#chirurgen
Nährstoff#aufnahme
Nikotin#assoziierte
Kar#zinome
Innenspangenre#sektion
Nikotin#assoziierte
Kar#zinome
Knochen#transplantation
Knochen#transplantation
Schluck#funktion
Schluck#funktion
Mund#bodenkarzinom
Chemotherapie#bedingte
Chemotherapie#bedingte
Radio#chemotherapie
Radio#chemotherapie
Mund#bodenkarzinoms
Knochen#nekrosen
Nerv#infiltration
Radio#chemotherapie
Mund#pflege
ggf.
Geschmacks#störungen
Geschmacks#störungen
Schlund#muskulatur
Radio#chemotherapie
Radio#chemotherapie
Hals#haut
Hals#haut
Radio#chemotherapie
Radio#chemotherapie
Radio#chemotherapie
Mund#höhlenkarzinomen
Mund#höhlenkarzinomen
Leistungs#zustand
Immun#zellen
Immun#zellen
Leistungs#zustand
Leistungs#zustand
Gesamt#überleben
Konvention#ell
Radio#chemotherapie
Schluck#störungen
Schluck#störungen
Körper#haltung
Schluck#störung
Schluck#störung
Behandlungs#bedingten
Beeinträchtig#ungen
Schl

# Reconstruct Complete Sentences

In [17]:
dom = minidom.parse(open(path_output_file))
predictions = []

sentences = dom.getElementsByTagName('s')
for sentence in sentences:
    tokens = sentence.getElementsByTagName('w')
    sentence_tokens = []
    for token in tokens:
        word = token.childNodes[0].nodeValue
        lemma = token.getAttribute('lemma')
        if '+' in lemma:
            word_parts = lemma.partition('+')
            if token.getAttribute('pos') == 'TRUNC':
                word = word.replace('-', word_parts[2])
            else:
                word = word.replace('-', word_parts[0])
        whitespace = token.getAttribute('whitespace')
        sentence_tokens.append(word + whitespace)

    predictions.append(''.join(token for token in sentence_tokens))

# Evaluate Baseline

In [19]:
errors_df = error_analysis(predictions, data.full_resolution, data.raw_sentence)
errors_df.error_type.value_counts() / len(errors_df)

tp         0.689709
fn         0.200494
replace    0.041970
delete     0.031575
complex    0.021700
insert     0.013903
fp         0.000650
Name: error_type, dtype: float64

In [26]:
%%time
scores = get_scores(errors_df, "eval")
scores

CPU times: user 8min 28s, sys: 321 ms, total: 8min 28s
Wall time: 8min 31s


{'eval/tp': 0.6897089397089398,
 'eval/tp_abs': 5308,
 'eval/fn': 0.20049376299376298,
 'eval/fn_abs': 1543,
 'eval/fp': 0.0006496881496881497,
 'eval/fp_abs': 5,
 'eval/replace': 0.04196985446985447,
 'eval/replace_abs': 323,
 'eval/insert': 0.013903326403326404,
 'eval/insert_abs': 107,
 'eval/delete': 0.03157484407484407,
 'eval/delete_abs': 243,
 'eval/complex': 0.0216995841995842,
 'eval/complex_abs': 167,
 'eval/edit_distance_rel': 0.7660947866445874,
 'eval/exact_match': 0.6897089397089398,
 'eval/gleu': 0.9627338689499875}

In [27]:
from notebook_util import show_errors

In [28]:
errors = pd.concat([errors_df, data[['file', 'sentence_id']].reset_index()], axis=1)

In [29]:
fps = errors[errors.error_type == 'fp'][['file', 'sentence_id', 'ground_truth', 'pred']].rename(columns = {'ground_truth' : 'resolution'})
print(fps.shape)
#fps.to_excel('../potential_missing_2.xlsx')

(5, 4)


In [ ]:
show_errors(errors[errors.error_type == 'fn'])

0;00_mundhoehlenkarzinom_0005.tsv;12

__Input:__

Die hohe Diskrepanz zwischen p16 Immunhistochemie bzw. HPV-RNA und HPV-DNA Nachweis zeigt, dass HPV-DNA in dieser Lokalisation zwar häufig nachweisbar ist, jedoch in der Mehrheit der Fälle wahrscheinlich nicht biologisch aktiv ist ( [REF] ).

__Error type:__ fn

1;00_mundhoehlenkarzinom_0005.tsv;18

...

__Error type:__ fn

580;09_mammakarzinom_0117.tsv;1

__Input:__

Das therapeutische Konzept bei Risikoläsionen soll nach Vorliegen des histologischen Befundes aus einer Stanz-/Vakuumbiopsie interdisziplinär (Radiodiagnostik, Operateur, Pathologie) erstellt werden.

__Error type:__ fn